In [1]:
import base64
import datetime
import json
import pandas as pd
import pickle
import pprint
import numpy as np
import requests
import time

# Setup Configuration

In [2]:
client_key = 'k7IeaeoVVSVVRs2VMRyjWMGhB'
client_secret = 'QjWhmf3ylIKXTYn6zv26tTkPjobjbapTKlC4JDay74jd647mcl'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAO3%2BQgEAAAAAKO%2BcZLOP%2B1jh1SeWDdIMpCF4smc%3DiUFDhj7SZfb1fl177n5Fipd9OA2Elj9aVdezk2hhBUGbTrdLgY'

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

base_url = 'https://api.twitter.com/'
auth_endpoint = base_url + 'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8' }

auth_data = { 'grant_type': 'client_credentials' }

# Get access token
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [3]:
json_data =  response.json()
access_token = json_data['access_token']

## Setup Request Params

In [4]:
seven_days_ago_now = datetime.datetime.now() - datetime.timedelta(days=6)

# Conduct Sample Request

In [98]:
max_tweets = 200
tweet_limit = 100

search_url = base_url + '2/tweets/search/recent'

# Seed Originating Tweets

In [93]:
def get_tweets(access_token, query, max_tweets=10, tweet_limit=10):
    """Retrieve tweets from the recent search API.
        Args
        ----
        access_token (str): A valid bearer token for making Twitter API requests.
        query (str): A valid Twitter query string for filtering the search tweets.
        max_tweets (int): The maximum number of tweets to collect in total.
        tweet_limit (int): The number of maximum tweets per API request. 
    """

    page_token = None
    tweet_data = []

    search_headers = {
        'Authorization': 'Bearer {}'.format(access_token),
        'User-Agent': 'v2FullArchiveSearchPython',
    }

    # Divides the max tweets into the appropriate number of requests based on the tweet_limit.
    for i in range(max_tweets // tweet_limit - 1):
        search_parameters = {
            'query': query,
            'max_results': tweet_limit,
            'tweet.fields': 'lang,created_at,referenced_tweets,source,conversation_id'
        }

        print(f'Request {i + 1}: {query}')
        
        # If we reach the 2nd page of results, add a next_token attribute to the search parameters
        if i > 0:
            search_parameters['next_token'] = page_token

        response = requests.get(search_url, headers=search_headers, params=search_parameters)
        if response.status_code != 200:
            print(f'\tError occurred: Status Code{response.status_code}: {response.text}')
        else:
            # We need to check for a result count before doing anything futher; if we have result_count we have data
            if response.json()['meta']['result_count'] > 0:
                tweet_data.extend(response.json()['data'])
                
                # If a 'next_token' exists, then update the page token to continue pagination through results
                if 'next_token' in response.json()['meta']:
                    page_token = response.json()['meta']['next_token']
            else:
                print(f'\tNo data returned for query!')
                break
        
        print(f'\t{len(tweet_data)} total tweets gathered')

        time.sleep(1)

    return tweet_data

In [99]:
# Make the request
q = '#olympics gymnastics -is:retweet'
olympic_tweets = get_tweets(access_token, q, max_tweets, tweet_limit)
# olympic_tweets

Request 1: #olympics gymnastics -is:retweet
	100 total tweets gathered


In [100]:
len(olympic_tweets)

100

In [101]:
with open('../data/olympic-gynmastics.pkl', 'wb') as f:
	pickle.dump(olympic_tweets, f)

# Deeper Conversations
In this section, we aim to gain additional context to Olympic tweets by digging down into the conversational elements of the Twitter-sphere: replies. 

In [102]:
gynmastics_tweets = pd.read_pickle("../data/olympic-gynmastics.pkl")

In [103]:
# Get the conversation IDs for the tweets
conversation_ids = [ tweet['conversation_id'] for tweet in gynmastics_tweets ]
len(conversation_ids)

100

In [104]:
# Query all replies
replies = []

for id in conversation_ids[0:11]:
	q = 'conversation_id:{id}'.format(id=id)
	replies.extend(get_tweets(access_token, q, max_tweets, tweet_limit))

Request 1: conversation_id:1423099685938466820
	No data returned for query!
Request 1: conversation_id:1423099132210675716
	No data returned for query!
Request 1: conversation_id:1423098788994961408
	No data returned for query!
Request 1: conversation_id:1423097976600342532
	No data returned for query!
Request 1: conversation_id:1423097895557926915
	No data returned for query!
Request 1: conversation_id:1423096463198810117
	No data returned for query!
Request 1: conversation_id:1423025757727625228
	100 total tweets gathered
Request 1: conversation_id:1423089600931635200
	1 total tweets gathered
Request 1: conversation_id:1422950455097786369
	18 total tweets gathered
Request 1: conversation_id:1423054645606457348
	8 total tweets gathered
Request 1: conversation_id:1423046468412264449
	11 total tweets gathered


In [ ]:
replies[0:11]

In [ ]:
with open('../data/olympic-gynmastics-replies.pkl', 'wb') as f:
	pickle.dump(replies, f)

# Inspect the Data
* Convert the original tweet data to a dataframe
* Convert the response tweet data to a dataframe

In [108]:
pd.set_option('expand_frame_repr', False)

In [109]:
original_tweets_df = pd.DataFrame(olympic_tweets)
original_tweets_df.head(5)

,id,source,created_at,text,conversation_id,lang,author_id,referenced_tweets
0,1423099685938466820,Twitter for Android,2021-08-05T01:52:59.000Z,#celebrity #celebrities #news #newstuff #News1...,1423099685938466820,und,1393325909344538627,NaN
1,1423099132210675716,Twitter for Android,2021-08-05T01:50:47.000Z,#celebrity #celebrities #news #newstuff #News1...,1423099132210675716,und,24902316,"[{'type': 'quoted', 'id': '1423095034983948297'}]"
2,1423098788994961408,Twitter for Android,2021-08-05T01:49:25.000Z,#celebrity #celebrities #news #newstuff #News1...,1423098788994961408,und,1393325909344538627,"[{'type': 'quoted', 'id': '1423095724212989952'}]"
3,1423097976600342532,Twitter for Android,2021-08-05T01:46:12.000Z,I miss gymnastics already #Olympics #Tokyo2020,1423097976600342532,en,2363192719,NaN
4,1423097895557926915,Twitter for Android,2021-08-05T01:45:52.000Z,This #Olympics #gymnastics floor routine is pr...,1423097895557926915,en,460754547,NaN


In [110]:
response_tweet_df = pd.DataFrame(replies)
response_tweet_df.head(5)

,text,lang,created_at,conversation_id,source,author_id,referenced_tweets,id
0,@Chris_Meloni https://t.co/9Uxk0kQbmh,und,2021-08-05T01:55:52.000Z,1423025757727625228,Twitter for iPad,3799025354,"[{'type': 'replied_to', 'id': '142302575772762...",1423100412744257540
1,@Chris_Meloni is there a pic from behind too?,en,2021-08-05T01:52:15.000Z,1423025757727625228,Twitter Web App,1109803723583340545,"[{'type': 'replied_to', 'id': '142302575772762...",1423099499774423043
2,@Chris_Meloni What’s with the popcorn? https:/...,en,2021-08-05T01:48:54.000Z,1423025757727625228,Twitter for iPhone,2937878247,"[{'type': 'replied_to', 'id': '142302575772762...",1423098658678005764
3,@Chris_Meloni 😍😍😍,und,2021-08-05T01:47:17.000Z,1423025757727625228,Twitter for iPhone,1393181802378973194,"[{'type': 'replied_to', 'id': '142302575772762...",1423098251587239946
4,@Chris_Meloni Are you an actor?,en,2021-08-05T01:43:58.000Z,1423025757727625228,Twitter for iPhone,294122390,"[{'type': 'replied_to', 'id': '142302575772762...",1423097417122074638


# Data Cleansing
* Convert data types
* Handle NaN values
* Clean text
	* Remove stop words
	* Remove puncutation
	* Remove user mentions
	* Remove hashtags
	* Remove URLs

# Word Preparation
* Convert casing
* Tokenize words
* Stemming & Lemmatization
* Check for stop words and determine how to handle
* POS tagging